In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

In [20]:
def cat_sbases(sbases):
    sbases=[torch.unsqueeze(sbase,1) for sbase in sbases]
    cats=torch.cat(sbases,1)
    return cats
def matmul(bases,attens):
    outs=None
    bases.transpose_(1,2)
    for i in range(bases.size(0)):
        out=torch.mm(bases[i],attens[i]).unsqueeze(0)
        if outs==None:
            outs=out
        else:
            outs=torch.cat(outs,out,0)
    return outs.squeeze()
class TLANet(nn.Module):
    def __init__(self,
                 input_shape,
                 num_tasks,
                 num_units_shared,
                 num_nuits_sbases,
                 num_units_experts,
                 num_sbases,
                 num_experts,
                 use_expert_b=True,
                 use_sbase_b=True,
                 use_l1att_b=True,
                 use_l2att_b=True,
                ):
        super(TLANet,self).__init__()
        self.num_tasks=num_tasks
        self.num_units_shared=num_units_shared
        self.num_nuits_sbases=num_nuits_sbases
        self.num_units_experts=num_units_experts
        self.num_experts=num_experts
        self.num_sbases=num_sbases
        self.use_expert_b=use_expert_b
        self.use_sbase_b=use_sbase_b
        self.use_l1att_b=use_l1att_b
        self.use_l2att_b=use_l2att_b
        
        #根据传入的参数定义层
        #共享基
        self.sbases=[]
        input_dimension=input_shape[-1]
        for i in range(num_sbases):
            if i ==0:
                self.sbases.append(nn.Linear(input_dimension,num_nuits_sbases,bias=use_sbase_b))
            else :
                self.sbases.append(nn.Linear(num_nuits_sbases,num_nuits_sbases,bias=use_sbase_b))
        
        self.shared=nn.Linear(num_nuits_sbases,num_units_shared,bias=use_sbase_b)
        self.experts=[nn.Linear(num_nuits_sbases,num_units_experts,bias=use_expert_b) for i in range(num_tasks)]
        self.l1atts=[nn.Linear(num_nuits_sbases,num_nuits_sbases,bias=use_l1att_b) for i in range(num_experts)]
        self.l2atts=[nn.Linear(num_units_experts,num_units_experts,bias=use_l2att_b) for i in range(num_tasks)]
        
        self.l1att2experts_ws=[nn.Linear(num_nuits_sbases,1) for i in range(num_experts)]
        self.l2att2tasks_ws=[nn.Linear(num_units_experts,1) for i in range(num_tasks)]
    def forward(self,x):
        sbs=[]
        experts=None
        for sbase in self.sbases:
            x=sbase(x)
            x=F.relu(x)
            sbs.append(x)
        x=self.shared(x)
        x=F.relu(x)
        experts=shared.unsqueeze(1)
        cat_sbases=cat_sbases(sbs)

        for i in range(self.num_experts):
            y=l1atts[i](cat_sbases)
            y=F.tanh(y)
            y=l1att2experts_ws[i](y)
            y=F.softmax(y,dim=1)
            y=matmul(cat_sbases,y)
            
            expert=self.experts[i](y)
            expert=F.relu(expert)
            if experts==None:
                experts=expert.unsqueeze(1)
            else:
                experts=torch.cat(experts,expert.unsqueeze(1),1)

        outputs=[]
        for i in range(self.num_tasks):
            z=l2atts[i](experts)
            z=F.tanh(z)
            z=l2att2tasks_ws[i](z)
            z=F.softmax(z,dim=1)
            z=matmul(experts,z)
            outputs.append(z)
        return outputs

            
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        

In [21]:
x=TLANet((100,200),3,10,10,10,10,10)

In [22]:
print(x)

TLANet(
  (shared): Linear(in_features=10, out_features=10, bias=True)
)
